In [5]:
import sys
sys.path.append("..")

from molsberry.core import Pipeline, InputBlock, OutputBlock
from molsberry.core import SDFPathRep, PDBPathRep
from molsberry.core import LigandData, ProteinData
from molsberry.modules.generic.math.blocks import Multiplier, Divider, Adder, Subtractor

from molsberry.modules.mopac import MOPACMozymeConfig
from molsberry.modules.mopac import (
    MOPACSinglePointCalculator, MOPACPLInteractionCalculator,
    MOPACLigandOptimizer
)

In [6]:
class BindingEnergyCalcPipe(Pipeline):
    name = "bindenpipe"
    display_name = "Binding Energy Calculation Pipeline"
    def build(self):
        config = MOPACMozymeConfig()
        config.keywords.extend(["PM6-D3H4X", "EPS=78.5", "VDW(C=1.821;N=1.904;O=1.682;H=0.828)", "NOMM"])
        self.add_block(InputBlock(["ligand"]), "input")
        self.add_block(MOPACLigandOptimizer(config, "LBFGS"), "ligopt")
        self.add_block(Subtractor(), 'substractor')
        self.add_block(OutputBlock(['num_out']), "output")
        self.add_connection("input", "ligand", "ligopt", "ligands")
        self.add_connection("ligopt", "e_init", "substractor", "num1")
        self.add_connection("ligopt", "e_final", "substractor", "num2")
        self.add_connection('substractor', 'num_out', 'output', 'num_out')


In [7]:
ligdata = LigandData(SDFPathRep("lig.sdf"))

In [8]:
pipeline = BindingEnergyCalcPipe(base_dir="./example1_out")
out = pipeline.execute(ligand=ligdata)


>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
>>   STARTED: Binding Energy Calculation Pipeline    >>
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

[Running Pipe Block: (ligopt) MOPAC Ligand Optimizer]
<class 'molsberry.core.data.molecules.LigandData'>
<molsberry.core.data.molecules.LigandData object at 0x7f101013cb50> <class 'molsberry.core.data.molecules.LigandData'>
[<class 'molsberry.modules.mopac.representations.MOPACInputMolRep'>, <class 'molsberry.modules.rdkit.representations.RDKitMolRep'>]
example1_out/GEDXY3.out
 *******************************************************************************
 **                                                                           **
 **                              MOPAC v23.0.1                                **
 **                                                                           **
 *******************************************************************************
 **          Digital Object Identifier (DOI):

In [9]:
out.items()

dict_items([('num_out', <molsberry.core.data.collections.BatchedData object at 0x7f1010147090>)])

In [10]:
out.items()

dict_items([('num_out', <molsberry.core.data.collections.BatchedData object at 0x7f1010147090>)])

In [11]:
{k: v.get_representation_content()[0] for k, v in out.items()}

{'num_out': 2.2492400000000004}

In [12]:
out['e_lig_init'].get_representation_content

KeyError: 'e_lig_init'

In [1]:
import sys
sys.path.append("..")

from molsberry.core import NumericData, FloatRep
from molsberry.modules.generic.math.blocks import Multiplier, Divider, Adder, Subtractor
from molsberry.core import Pipeline, InputBlock, OutputBlock

class Divide(Pipeline):
    name = "divider"
    display_name = "Divider Pilpeline"
    def build(self):
        self.add_block(InputBlock(["num1", "num2"]), "input")
        self.add_block(Divider(), "divider")
        self.add_block(OutputBlock(['num_out']), "output")
        self.add_connection("input", "num1", "divider", "num1")
        self.add_connection("input", "num2", "divider", "num2")
        self.add_connection("divider", "num_out", "output", "num_out")

num1 = NumericData(FloatRep(12.1))
num2 = NumericData(FloatRep(12))

pipeline = Divide(base_dir="./example1_out")
out = pipeline.execute(num1 = num1, num2 = num2)

TypeError: Can't instantiate abstract class Divider with abstract method inputs

In [2]:
num1 = NumericData(FloatRep(12))
num1.get_representation

<bound method Data.get_representation of <molsberry.core.data.generic.NumericData object at 0x7f505009d310>>

In [3]:
num1 = FloatRep(12.1)
num2 = FloatRep(12)
num1 + num2

24.1

In [4]:
out['num_out'].get_representation

<bound method BatchedData.get_representation of <molsberry.core.data.collections.BatchedData object at 0x7f4feb09d790>>

In [5]:
import sys
sys.path.append("..")

from molsberry.core import NumericData, FloatRep
from molsberry.modules.generic.math.blocks import Multiplier, Divider
from molsberry.core import Pipeline, InputBlock, OutputBlock

class Divide(Pipeline):
    name = "divider"
    display_name = "Divider Pilpeline"
    def build(self):
        self.add_block(InputBlock(["num1", "num2", 'num3']), "input")
        self.add_block(Divider(), "divider")
        self.add_block(Multiplier(), 'multiplier')
        self.add_block(OutputBlock(['num_out']), "output")
        self.add_connection("input", "num1", "divider", "num1")
        self.add_connection("input", "num2", "divider", "num2")
        self.add_connection("input", "num3", "multiplier", "num2")
        self.add_connection("divider", "num_out", "multiplier", "num1")
        self.add_connection('multiplier', 'num_out', 'output', 'num_out')
        

num1 = NumericData(FloatRep(12.1))
num2 = NumericData(FloatRep(12))
num3 = NumericData(FloatRep('14'))

pipeline = Divide(base_dir="./example1_out")
out = pipeline.execute(num1 = num1, num2 = num2, num3 = num3)


>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
>>            STARTED: Divider Pilpeline             >>
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

[Running Pipe Block: (divider) Numeric divider]
<class 'molsberry.core.data.generic.NumericData'>
<molsberry.core.data.generic.NumericData object at 0x7f4feb0c7f50> <class 'molsberry.core.data.generic.NumericData'>
<class 'molsberry.core.data.generic.FloatRep'>
<class 'molsberry.core.data.generic.NumericData'>
<molsberry.core.data.generic.NumericData object at 0x7f4feb0bc310> <class 'molsberry.core.data.generic.NumericData'>
<class 'molsberry.core.data.generic.FloatRep'>

[Running Pipe Block: (multiplier) Numeric Multiplier]
<class 'molsberry.core.data.generic.NumericData'>
<molsberry.core.data.generic.NumericData object at 0x7f4feb0bd250> <class 'molsberry.core.data.generic.NumericData'>
<class 'molsberry.core.data.generic.FloatRep'>
<class 'molsberry.core.data.generic.NumericData'>
<molsberry.core.data.generic.Numeri

In [6]:
out['num_out'].get_representation_content(FloatRep)[0]

14.116666666666667

In [1]:
import sys
sys.path.append("..")

from molsberry.core import NumericData, FloatRep
from molsberry.modules.generic.math.blocks import Multiplier, Divider, Adder
from molsberry.core import Pipeline, InputBlock, OutputBlock

class Add(Pipeline):
    name = "adder"
    display_name = "Adder Pilpeline"
    def build(self):
        self.add_block(InputBlock(["num1", "num2", 'num3']), "input")
        self.add_block(Adder(num_inputs=3), "adder")
        self.add_block(OutputBlock(['num_out']), "output")
        self.add_connection("input", "num1", "adder", "num1")
        self.add_connection("input", "num2", "adder", "num2")
        self.add_connection("input", "num3", "adder", "num3")
        self.add_connection('adder', 'num_out', 'output', 'num_out')
        

num1 = NumericData(FloatRep(12.1))
num2 = NumericData(FloatRep(12))
num3 = NumericData(FloatRep('14'))

pipeline = Add(base_dir="./example1_out")
out = pipeline.execute(num1 = num1, num2 = num2, num3 = num3)


>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
>>             STARTED: Adder Pilpeline              >>
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

[Running Pipe Block: (adder) Numeric Adder]
<class 'molsberry.core.data.generic.NumericData'>
<molsberry.core.data.generic.NumericData object at 0x7fc1e4104f50> <class 'molsberry.core.data.generic.NumericData'>
<class 'molsberry.core.data.generic.FloatRep'>
<class 'molsberry.core.data.generic.NumericData'>
<molsberry.core.data.generic.NumericData object at 0x7fc1a3309190> <class 'molsberry.core.data.generic.NumericData'>
<class 'molsberry.core.data.generic.FloatRep'>
<class 'molsberry.core.data.generic.NumericData'>
<molsberry.core.data.generic.NumericData object at 0x7fc1a087c8d0> <class 'molsberry.core.data.generic.NumericData'>
<class 'molsberry.core.data.generic.FloatRep'>

[Running Pipe Block: (output) Output Block]

<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
<<              ENDED: Adder Pilpeline    

In [1]:
import sys
sys.path.append("..")

from molsberry.core import NumericData, FloatRep, NpData, NpArrayRep
from molsberry.modules.generic.math.blocks import Multiplier, Divider, Adder, NumericArrayGenerator
from molsberry.core import Pipeline, InputBlock, OutputBlock

class Numnp(Pipeline):
    name = "adder"
    display_name = "Adder Pilpeline"
    def build(self):
        self.add_block(InputBlock(["num1", "num2", 'num3']), "input")
        self.add_block(NumericArrayGenerator(num_inputs=3), "adder")
        self.add_block(OutputBlock(['array_out']), "output")
        self.add_connection("input", "num1", "adder", "num1")
        self.add_connection("input", "num2", "adder", "num2")
        self.add_connection("input", "num3", "adder", "num3")
        self.add_connection('adder', 'array_out', 'output', 'array_out')
        

num1 = NumericData(FloatRep(12.1))
num2 = NumericData(FloatRep(12))
num3 = NumericData(FloatRep('14'))

pipeline = Numnp(base_dir="./example1_out")
out = pipeline.execute(num1 = num1, num2 = num2, num3 = num3)


>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
>>             STARTED: Adder Pilpeline              >>
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

[Running Pipe Block: (adder) Numeric Array Generator]
<class 'molsberry.core.data.generic.NumericData'>
<molsberry.core.data.generic.NumericData object at 0x7f83210ca7d0> <class 'molsberry.core.data.generic.NumericData'>
<class 'molsberry.core.data.generic.FloatRep'>
<class 'molsberry.core.data.generic.NumericData'>
<molsberry.core.data.generic.NumericData object at 0x7f83210c4b50> <class 'molsberry.core.data.generic.NumericData'>
<class 'molsberry.core.data.generic.FloatRep'>
<class 'molsberry.core.data.generic.NumericData'>
<molsberry.core.data.generic.NumericData object at 0x7f831fb1dd90> <class 'molsberry.core.data.generic.NumericData'>
<class 'molsberry.core.data.generic.FloatRep'>

[Running Pipe Block: (output) Output Block]

<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
<<              ENDED: Adder Pil

In [3]:
out['array_out'].get_representation_content()[0]

array([12.1, 12. , 14. ])

In [2]:
import sys
sys.path.append("..")

from molsberry.core import NumericData, FloatRep, NpData, NpArrayRep
from molsberry.modules.generic.math.blocks import Multiplier, Divider, Adder, NumericArrayGenerator, CorrCoef, SKLinearRegression, SKPredictor
from molsberry.core import Pipeline, InputBlock, OutputBlock

class LR(Pipeline):
    name = "lr"
    display_name = "linear regression Pilpeline"
    def build(self):
        self.add_block(InputBlock(["npa1", "npa2", 'num']), "input")
        self.add_block(SKLinearRegression(), "lr")
        self.add_block(SKPredictor(), 'sp')
        self.add_block(OutputBlock(['model']), "output")
        self.add_connection("input", "npa1", "lr", "npa1")
        self.add_connection("input", "npa2", "lr", "npa2")
        #self.add_connection('lr', 'model', 'sp', 'model')
        #self.add_connection('input', 'num', 'sp', 'num')
        self.add_connection('lr', 'model', 'output', 'model')

npa1 = NpData(NpArrayRep([12, 123]))
npa2 = NpData(NpArrayRep([12, 123]))
num = NumericData(FloatRep(14))

pipeline = LR(base_dir="./example1_out")
out = pipeline.execute(npa1 = npa1, npa2 = npa2, num = num)


>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
>>       STARTED: linear regression Pilpeline        >>
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

[Running Pipe Block: (lr) sk-learn linear regression model]
<class 'molsberry.core.data.generic.NpData'>
<molsberry.core.data.generic.NpData object at 0x7f1e3ea1ed90> <class 'molsberry.core.data.generic.NpData'>
<class 'molsberry.core.data.generic.NpArrayRep'>
<class 'molsberry.core.data.generic.NpData'>
<molsberry.core.data.generic.NpData object at 0x7f1deb02c4d0> <class 'molsberry.core.data.generic.NpData'>
<class 'molsberry.core.data.generic.NpArrayRep'>

[Running Pipe Block: (output) Output Block]

<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
<<        ENDED: linear regression Pilpeline         <<
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<


In [3]:
out['model'].get_representation_content()[0]

'example1_out/MWANYF.pkl'

In [6]:
out['model'].get_representation_content()

In [1]:
import sys
sys.path.append("..")

from molsberry.core import NumericData, FloatRep, NpData, NpArrayRep, StringData, StringRep
from molsberry.modules.generic.math.blocks import Multiplier, Divider, Adder, NumericArrayGenerator, CorrCoef, SKLinearRegression, SKPredictor
from molsberry.core import Pipeline, InputBlock, OutputBlock

class LR(Pipeline):
    name = "lr"
    display_name = "linear regression Pilpeline"
    def build(self):
        self.add_block(InputBlock(['model', 'num']), "input")
        self.add_block(SKLinearRegression(), "lr")
        self.add_block(SKPredictor(), 'sp')
        self.add_block(OutputBlock(['num_out']), "output")
        #self.add_connection("input", "npa1", "lr", "npa1")
        #self.add_connection("input", "npa2", "lr", "npa2")
        self.add_connection('input', 'model', 'sp', 'model')
        self.add_connection('input', 'num', 'sp', 'num')
        self.add_connection('sp', 'num_out', 'output', 'num_out')

npa1 = NpData(NpArrayRep([12, 123]))
npa2 = NpData(NpArrayRep([12, 123]))
num = NumericData(FloatRep(14))
model = StringData(StringRep('example1_out/MWANYF.pkl'))

pipeline = LR(base_dir="./example1_out")
out = pipeline.execute(model = model, num = num)


>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
>>       STARTED: linear regression Pilpeline        >>
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

[Running Pipe Block: (sp) Sk-learn predictor with the model]
<class 'molsberry.core.data.generic.StringData'>
<molsberry.core.data.generic.StringData object at 0x7f8290546790> <class 'molsberry.core.data.generic.StringData'>
<class 'molsberry.core.data.generic.StringRep'>
<class 'molsberry.core.data.generic.NumericData'>
<molsberry.core.data.generic.NumericData object at 0x7f82ac1d6bd0> <class 'molsberry.core.data.generic.NumericData'>
<class 'molsberry.core.data.generic.FloatRep'>

[Running Pipe Block: (output) Output Block]

<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
<<        ENDED: linear regression Pilpeline         <<
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<


In [2]:
out['num_out'].get_representation_content()[0]

14.000000000000023

In [1]:
import sys
sys.path.append("..")

from molsberry.core import Pipeline, InputBlock, OutputBlock, MoleculeData, SDFPathRep
from molsberry.core import SDFPathRep, PDBPathRep
from molsberry.core import LigandData, ProteinData
from molsberry.core.data.collections import BatchedData

from molsberry.modules.generic.math.blocks import Multiplier, Divider, Adder, Subtractor

from molsberry.modules.mopac import MOPACMozymeConfig
from molsberry.modules.mopac import (
    MOPACSinglePointCalculator, MOPACPLInteractionCalculator,
    MOPACLigandOptimizer
)

class SF(Pipeline):
    name = "scoring function"
    display_name = "scoring function Pilpeline"
    def build(self):
        config = MOPACMozymeConfig()
        config.keywords.extend(["THREADS=8", "PM6-D3H4X", "NOMM", "EPS=78.5"])
        self.add_block(InputBlock(['ligand', 'protein']), "input")
        self.add_block(MOPACLigandOptimizer(config, "LBFGS", num_workers=4), "ligopt")
        self.add_block(MOPACPLInteractionCalculator(config), "intercalc")
        self.add_block(MOPACSinglePointCalculator(config), 'singlepoint')
        self.add_block(MOPACSinglePointCalculator(config), 'singlepoint2')
        self.add_block(Adder(num_inputs=2), 'add')
        self.add_block(Subtractor(), 'minus')
        self.add_block(OutputBlock(["results"]), "output")
        self.add_connection('input', 'protein', 'intercalc', 'proteins')
        self.add_connection('input', 'ligand', 'intercalc', 'ligands')
        self.add_connection('input', 'ligand', 'ligopt', 'ligands')
        self.add_connection('intercalc', 'e_interaction', 'add', 'num1')
        self.add_connection('input', 'ligand', 'singlepoint', 'molecules')
        self.add_connection('singlepoint', 'energy', 'minus', 'num1')
        self.add_connection('ligopt', 'ligands', 'singlepoint2', 'molecules')
        self.add_connection('singlepoint2', 'energy', 'minus', 'num2')
        self.add_connection('minus', 'num_out', 'add', 'num2')
        self.add_connection('add', 'num_out', 'output', 'results')

import glob
ligand_paths = glob.glob('/mnt/e/molsberry/PL-REX/004-AR/structures_pl-rex/*/ligand.sdf') #4 is a sample of errors
ligs = [LigandData(SDFPathRep(ligand)) for ligand in ligand_paths]
ligand = BatchedData(ligs)
protein_paths = glob.glob('/mnt/e/molsberry/PL-REX/004-AR/structures_pl-rex/*/receptor.pdb')
protein = [ProteinData(PDBPathRep(protein)) for protein in protein_paths]
protein = BatchedData(protein)

pipeline = SF(base_dir="./example1_out")
out = pipeline.execute(ligand = ligand, protein = protein)


>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
>>       STARTED: scoring function Pilpeline         >>
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

[Running Pipe Block: (intercalc) MOPAC Protein Ligand Interaction Calculator]
<class 'molsberry.core.data.molecules.LigandData'>
<molsberry.core.data.molecules.LigandData object at 0x7f7e985e2ad0> <class 'molsberry.core.data.molecules.LigandData'>
<class 'molsberry.modules.mopac.representations.MOPACInputMolRep'>
<class 'molsberry.core.data.molecules.ProteinData'>
<molsberry.core.data.molecules.ProteinData object at 0x7f7e522f2e10> <class 'molsberry.core.data.molecules.ProteinData'>
<class 'molsberry.modules.mopac.representations.MOPACInputMolRep'>
<class 'molsberry.core.data.molecules.LigandData'>
<molsberry.core.data.molecules.LigandData object at 0x7f7e52270a50> <class 'molsberry.core.data.molecules.LigandData'>
<class 'molsberry.modules.mopac.representations.MOPACInputMolRep'>
<class 'molsberry.core.data.molecules.

IndexError: list index out of range

In [2]:
for i in range(len(out['results'].get_representation_content())):
    print(out['results'].get_representation_content()[i])

NameError: name 'out' is not defined

In [3]:
import sys
sys.path.append("..")
from molsberry.core import NumericData, NpArrayRep, NpData
from molsberry.modules.generic.sklearn import CorrCoef
import numpy as np
#5
array1 = NpData(NpArrayRep(np.array([-57.90100000000052,
-60.8097099999996,
-62.536420000000106,
-59.4639600000005,
-55.077870000000445,
-57.68350999999893,
-64.3873899999993,
-56.91363000000052,
-54.45069999999947,
-41.85294999999951])))
array2 = NpData(NpArrayRep(np.array([-13.072,
-12.221,
-11.720,
-10.249,
 -9.646,
 -8.954,
-12.711,
 -9.067,
 -8.746,
 -8.236])))


class Correlation(Pipeline):
    name = "correlation coefficient"
    display_name = "Correlation CoefficientPilpeline"
    def build(self):
        self.add_block(InputBlock(['array1', 'array2']), "input")
        self.add_block(CorrCoef(), 'corrcoef')
        self.add_block(OutputBlock(["result"]), "output")
        self.add_connection('input', 'array1', 'corrcoef', 'npa1')
        self.add_connection('input', 'array2', 'corrcoef', 'npa2')
        self.add_connection('corrcoef', 'num_out', 'output', 'result')

pipeline = Correlation(base_dir="./example1_out")
out = pipeline.execute(array1 = array1, array2 = array2)


>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
>>    STARTED: Correlation CoefficientPilpeline      >>
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

[Running Pipe Block: (corrcoef) CorrCoeff calculator]
<class 'molsberry.core.data.generic.NpData'>
<molsberry.core.data.generic.NpData object at 0x7f7ead1ccd10> <class 'molsberry.core.data.generic.NpData'>
<class 'molsberry.core.data.generic.NpArrayRep'>
<class 'molsberry.core.data.generic.NpData'>
<molsberry.core.data.generic.NpData object at 0x7f7ead1e8dd0> <class 'molsberry.core.data.generic.NpData'>
<class 'molsberry.core.data.generic.NpArrayRep'>

[Running Pipe Block: (output) Output Block]

<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
<<     ENDED: Correlation CoefficientPilpeline       <<
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<


In [4]:
for i in range(len(out['results'].get_representation_content())):
    print(out['results'].get_representation_content()[i])

-42.3296399999996


In [2]:
import sys
sys.path.append("..")

from molsberry.core import Pipeline, InputBlock, OutputBlock, MoleculeData, SDFPathRep, NumericData, FloatRep
from molsberry.core import SDFPathRep, PDBPathRep
from molsberry.core import LigandData, ProteinData
from molsberry.core.data.collections import BatchedData

from molsberry.modules.generic.math.blocks import Multiplier, Divider, Adder, Subtractor

from molsberry.modules.mopac import MOPACMozymeConfig
from molsberry.modules.mopac import (
    MOPACSinglePointCalculator, MOPACPLInteractionCalculator,
    MOPACLigandOptimizer
)

class SF(Pipeline):
    name = "scoring function"
    display_name = "scoring function Pilpeline"
    def build(self):
        self.add_block(InputBlock(['num1', 'num2']), "input")
        self.add_block(Subtractor(), 'minus')
        self.add_block(OutputBlock(["results"]), "output")
        self.add_connection('input', 'num1', 'minus', 'num1')
        self.add_connection('input', 'num2', 'minus', 'num2')
        self.add_connection('minus', 'num_out', 'output', 'results')


#ligand = LigandData(SDFPathRep('/mnt/e/molsberry/molsberry/ligand (3).sdf'))
#protein = ProteinData(PDBPathRep('/mnt/e/molsberry/molsberry/receptor (1).pdb'))

num1 = [NumericData(FloatRep(12)), NumericData(FloatRep(13))]
num2 = [NumericData(FloatRep(14)), NumericData(FloatRep(15))]
num1 = BatchedData(num1)
num2 = BatchedData(num2)

pipeline = SF(base_dir="./example1_out")
out = pipeline.execute(num1 = num1, num2 = num2)

NameError: name 'num2' is not defined